---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import re

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [66]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    import re

    textfile = open('university_towns.txt', 'r')
    df = []    

    for row in textfile:
        rowText = row
    #     print(rowText)
        if 'edit' in rowText:
            stateName = re.sub('\[.*\]\n', '', rowText).strip()
        else:
            townName = re.sub('\n', ' ', rowText)    
            townName = re.sub('\(.*\]', '', townName)
            townName = re.sub('\(.*\)', '', townName).strip()          
            df.append({'State' : stateName, 'RegionName' : townName})    
            
    df = pd.DataFrame(df, columns = ['State', 'RegionName'])
#     df = pd.DataFrame(df)
#     df.reset_index(inplace=True)
#     df.set_index(['State', 'RegionName'], inplace=True)
#     df.drop(labels='index', axis=1, inplace=True)
    return df

get_list_of_university_towns().head()
# get_list_of_university_towns().RegionName



,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


In [18]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls', header=5, index_col=4)
    gdp = gdp.loc[gdp.index > '1999q4', :]
    
    # throw out columns we don't need
    gdp.drop(['GDP in billions of current dollars', 
              'GDP in billions of chained 2009 dollars',
             'Unnamed: 2',
             'Unnamed: 3',
             'Unnamed: 6'], axis=1, inplace=True)
    gdp.columns = ['GDP_current', 'GDP_2009']
    
    # if difference is <0, then next quarter is higher than previous quarter (i.e. no sign of recession)
    gdp['Diff'] = gdp['GDP_current'] - gdp['GDP_current'].shift(-1)
    # flag where GDP DECREASES in consecutive year as 1
    gdp['Flag'] = (gdp['Diff'] > 0).astype(int)
#     gdp['Flag2'] = gdp['Flag'] + gdp['Flag'].shift(2)
    
    gdp['Flag2'] = ((gdp['Flag'] == 1) & 
                    (gdp['Flag'].shift(-1) == 1) &
                   gdp['Flag'].shift(-2) == 1).astype(int)
    
    recession_start = gdp.index[gdp['Flag2'] == 1][0]
    return recession_start

get_recession_start()

'2008q3'

In [72]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    recession_start = get_recession_start()
    
    gdp = pd.read_excel('gdplev.xls', header=5, index_col=4)
    gdp = gdp.loc[gdp.index > recession_start, :]
    
    # throw out columns we don't need
    gdp.drop(['GDP in billions of current dollars', 
              'GDP in billions of chained 2009 dollars',
             'Unnamed: 2',
             'Unnamed: 3',
             'Unnamed: 6'], axis=1, inplace=True)
    gdp.columns = ['GDP_current', 'GDP_2009']
    
    # if difference is <0, then next quarter is higher than previous quarter (i.e. no sign of recession)
    gdp['Diff'] = gdp['GDP_current'] - gdp['GDP_current'].shift(-1)
    
    # flag where GDP INCREASES in consecutive year as 1
    gdp['Flag'] = (gdp['Diff'] < 0).astype(int)
    gdp['Flag2'] = ((gdp['Flag'] == 1) & 
                    (gdp['Flag'].shift(1) == 1) &
                   gdp['Flag'].shift(2) == 1).astype(int)
    
    recession_end = gdp.index[gdp['Flag2'] == 1][0]
    return recession_end

get_recession_end()

'2009q4'

In [73]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    recession_start = get_recession_start()
    recession_end = get_recession_end()
    
    gdp = pd.read_excel('gdplev.xls', header=5, index_col=4)
    gdp = gdp.loc[gdp.index > recession_start, :]
    gdp = gdp.loc[gdp.index < recession_end, :]
    
    # throw out columns we don't need
    gdp.drop(['GDP in billions of current dollars', 
              'GDP in billions of chained 2009 dollars',
             'Unnamed: 2',
             'Unnamed: 3',
             'Unnamed: 6'], axis=1, inplace=True)
    gdp.columns = ['GDP_current', 'GDP_2009']   
    
    recession_bottom = gdp.index[gdp['GDP_current'] == gdp['GDP_current'].min()][0]
    
    return recession_bottom

get_recession_bottom()

'2009q2'

In [19]:
def convert_housing_data_to_quarters():
    import numbers
    
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    housing = pd.read_csv('City_Zhvi_AllHomes.csv')
    
    # convert each month to its respective year and quarter
    col_names = [re.sub('-(01|02|03)', 'q1', string) for string in housing.columns]
    col_names = [re.sub('-(04|05|06)', 'q2', string) for string in col_names]
    col_names = [re.sub('-(07|08|09)', 'q3', string) for string in col_names]
    col_names = [re.sub('-(10|11|12)', 'q4', string) for string in col_names]
    housing.columns = col_names

    # group dataframe into multi-index columns with "State" and "RegionName" as categories
    housing['State'] = [states[state] for state in housing['State']]    
    housing.set_index(['State', 'RegionName'], inplace=True)
    housing.sort_index(inplace=True)    

    # find start and end indices of unwanted columns and drop
    c1 = np.max((housing.columns == 'RegionID').nonzero())
    c2 = np.max((housing.columns == '1999q4').nonzero())
    housing.drop(housing.columns[range(c1,c2)], axis=1, inplace=True)    

    # take the mean of all 4 quarters in each year
    # 1st way to do this
#     housing_data = housing.groupby(by=housing.columns, axis=1).apply(
#         lambda g: g.mean(axis=1) if isinstance(g.iloc[0,0], numbers.Number) else g.iloc[:,0])

    # 2nd way to do this
    housing_data = housing.mean(axis=1, level=0)

    return housing_data

convert_housing_data_to_quarters().index.get_level_values(1)

Index(['Adamsville', 'Alabaster', 'Albertville', 'Arab', 'Ardmore', 'Axis',
       'Baileyton', 'Bay Minette', 'Bayou La Batre', 'Bessemer',
       ...
       'Wolf River', 'Woodruff', 'Woodville', 'Yorkville', 'Bar Nunn', 'Burns',
       'Casper', 'Cheyenne', 'Evansville', 'Pine Bluffs'],
      dtype='object', name='RegionName', length=10730)

In [101]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # get relevant quarters and housing data
#     uni_towns = get_list_of_university_towns()
    uni_towns = get_list_of_university_towns()['RegionName']
    uni_towns = set(uni_towns)
    recession_start = get_recession_start()
    recession_end = get_recession_end()
    recession_bottom = get_recession_bottom()
    housing_data = convert_housing_data_to_quarters()
    housing_data.reset_index(inplace=True)
    
    def get_quarter_before_recession():
        start_quarter = [i for i, column_name in enumerate(housing_data.columns) if column_name == recession_start] 
        quarter_before_recession = housing_data.iloc[:, [start_quarter[0]-1]].columns.tolist()
        return quarter_before_recession
    quarter_before_recession = get_quarter_before_recession()

    def price_ratio(row):
        # price_ratio = quarter_before_recession / recession_bottom
        return row[quarter_before_recession]/row[recession_bottom]    
    housing_data['up&down'] = housing_data.apply(price_ratio, axis=1)
    
    def is_uni_town(row):
        # check if the town is an university town or not
        if row['RegionName'] in uni_towns:
            return 1
        else:
            return 0
    housing_data['is_uni'] = housing_data.apply(is_uni_town, axis=1)
    
    not_uni = housing_data[housing_data['is_uni']==0].loc[:,'up&down'].dropna()
    is_uni = housing_data[housing_data['is_uni']==1].loc[:,'up&down'].dropna()
    
    def better():
        if not_uni.mean() < is_uni.mean():
            return 'non-university town' 
        else:
            return 'university town'
    
    # run t-test
    p_val = list(ttest_ind(not_uni, is_uni))[1]
    if p_val < 0.01:
        different = True
    else:
        different = False
        
    result = (different, p_val, better())
        
    return result


run_ttest()

(True, 3.4411540629710297e-05, 'university town')